In [10]:
import numpy as np

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivate(x):
    return x * (1.0 - x)

inputLayer, hiddenLayer, outputLayer = 2,2,1

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])


w_1 = np.random.uniform(size=(inputLayer, hiddenLayer))
w_2 = np.random.uniform(size=(hiddenLayer, outputLayer))


output_1 = sigmoid(np.dot(X, w_1))
output_2 = sigmoid(np.dot(output_1, w_2))

error = Y - output_2

